## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-30-01-10-54 +0000,nypost,New York Times stunningly rolls back claims ab...,https://nypost.com/2025/07/29/media/new-york-t...
1,2025-07-30-01-07-38 +0000,nyt,Grassley Spurns Whistle-Blowers in Bove Confir...,https://www.nytimes.com/2025/07/29/us/politics...
2,2025-07-30-01-07-18 +0000,nyt,"Senate, Rejecting Whistle-Blower Alarms, Confi...",https://www.nytimes.com/2025/07/29/us/politics...
3,2025-07-30-01-05-31 +0000,latimes,"Tsunami watch for California: What we know, ho...",https://www.latimes.com/california/story/2025-...
4,2025-07-30-01-02-00 +0000,wsj,Possible dissents this week by Christopher Wal...,https://www.wsj.com/economy/central-banking/fe...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2312/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
93,trump,59
0,new,22
9,gaza,19
86,shooting,17
84,nyc,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
72,2025-07-29-22-00-58 +0000,nyt,"Trump Says Gaza Will Get More Aid, but Israel ...",https://www.nytimes.com/2025/07/29/us/politics...,122
280,2025-07-29-09-30-00 +0000,wsj,Capital Account: Trump has hit his tariff goal...,https://www.wsj.com/economy/trade/trumps-new-t...,121
33,2025-07-29-23-46-00 +0000,wsj,The Trump administration has put a halt on bil...,https://www.wsj.com/politics/policy/trump-admi...,117
165,2025-07-29-17-49-06 +0000,nypost,"Moscow taunts Trump by comparing him to Biden,...",https://nypost.com/2025/07/29/world-news/mosco...,113
168,2025-07-29-17-38-03 +0000,nypost,UN-backed experts warn of ‘worst-case scenario...,https://nypost.com/2025/07/29/world-news/un-ba...,109


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
72,122,2025-07-29-22-00-58 +0000,nyt,"Trump Says Gaza Will Get More Aid, but Israel ...",https://www.nytimes.com/2025/07/29/us/politics...
98,77,2025-07-29-21-18-23 +0000,wapo,New details emerge in New York City shooting i...,https://www.washingtonpost.com/nation/2025/07/...
275,69,2025-07-29-09-50-34 +0000,cbc,Gunman who killed 4 at NYC building was target...,https://www.cbc.ca/news/world/gunman-manhattan...
209,68,2025-07-29-15-30-51 +0000,nypost,Union Pacific strikes $85B deal to buy rival N...,https://nypost.com/2025/07/29/business/union-p...
316,54,2025-07-29-01-45-28 +0000,nypost,Extension of US-China tariff pause ‘likely’ – ...,https://nypost.com/2025/07/28/us-news/extensio...
130,47,2025-07-29-19-52-56 +0000,nyt,UK Will Recognize Palestinian Statehood In Sep...,https://www.nytimes.com/2025/07/29/world/europ...
80,37,2025-07-29-21-42-41 +0000,nypost,Top Zelensky official praises Trump for giving...,https://nypost.com/2025/07/29/us-news/top-zele...
33,36,2025-07-29-23-46-00 +0000,wsj,The Trump administration has put a halt on bil...,https://www.wsj.com/politics/policy/trump-admi...
41,36,2025-07-29-23-16-46 +0000,nypost,Massacre at Park Avenue office tower by derang...,https://nypost.com/2025/07/29/business/massacr...
243,32,2025-07-29-13-00-00 +0000,wsj,"Risks from the trade war remain, but the road ...",https://www.wsj.com/economy/tariff-detente-lif...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
